## 基本方程

这个笔记本介绍了用于求解一维弹性波方程的有限元代码。另外，还提供了用有限差分方案求解的解来进行比较。

解决波动方程问题时，首先对整个物理区域 $D$ 进行积分，并同时乘以一些基函数 $\varphi_{i}$，从而得到波动方程的弱形式。

接着进行积分分部运算，并实现无应力边界条件。

我们用时变系数 $u_i(t)$ 加权的空间依赖基函数 $\varphi_i$ 的和来近似未知的位移场 $u(x, t)$。

\begin{equation}
u(x,t) \ \approx \ \overline{u}(x,t) \ = \ \sum_{i=1}^{n} u_i(t) \ \varphi_i(x)
\end{equation}

然后，在弱形式中使用与展开 $u(x, t)$ 的相同基函数作为测试函数，这就是 Galerkin 原理。

通过考虑近似位移场，我们可以将连续的弱形式转换为线性方程组。

\begin{equation}
\mathbf{M}^T\partial_t^2 \mathbf{u} + \mathbf{K}^T\mathbf{u} = \mathbf{f}
\end{equation}

对于二阶时间导数，我们使用标准的有限差分逼近。最终，我们得到了显式时间外推方案。

\begin{equation}
\mathbf{u}(t + dt) = dt^2 (\mathbf{M}^T)^{-1}[\mathbf{f} - \mathbf{K}^T\mathbf{u}] + 2\mathbf{u} - \mathbf{u}(t-dt)
\end{equation}

其中，$\mathbf{M}$ 被称为质量矩阵，$\mathbf{K}$ 是刚度矩阵。

我们选择插值函数作为插值函数，使得 $\varphi_{i}(x_{i}) = 1$，其他地方为零。然后，我们将空间坐标转换为本地坐标系。根据 $\xi = x − x_{i}$ 和 $h_{i} = x_{i+1} − x_{i}$，我们有：

\begin{equation}
 \varphi_{i}(\xi) =
  \begin{cases}
    \frac{\xi}{h_{i-1}} + 1  & \quad \text{if} \quad -h_{i-1} \le \xi \le 0\\
    1 + \frac{\xi}{h_{i}}    & \quad \text{if} \quad 0 \le \xi \le h_{i}\\
    0  & \quad elsewhere\\
  \end{cases}
\end{equation}

对应的导数为

\begin{equation}
 \partial_{\xi}\varphi_{i}(\xi) =
  \begin{cases}
    \frac{1}{h_{i-1}}  & \quad \text{if} \quad -h_{i-1} \le \xi \le 0\\
   -\frac{1}{h_{i}}    & \quad \text{if} \quad 0 \le \xi \le h_{i}\\
    0  & \quad elsewhere\\
  \end{cases}
\end{equation}

左侧的图示了 $\varphi_{i}(\xi)$ 和 $\partial_{\xi}\varphi_{i}(\xi)$ 随着 $h$ 变化的形状。

代码实现从问题设置的初始化开始。然后，我们定义引入扰动的源，然后初始化质量矩阵和刚度矩阵。最后，进行时间外推。


In [2]:
# Import all necessary libraries, this is a configuration step for the exercise.
# Please run it before the simulation code!
%matplotlib inline
import numpy as np
import matplotlib
# Show Plot in The Notebook
matplotlib.use("nbagg")
import matplotlib.pyplot as plt
import matplotlib.animation as animation


In [3]:
# Initialization of setup
# ---------------------------------------------------------------
# Basic parameters
nx    = 1000    # Number of collocation points  
xmax  = 10000.  # Physical dimension [m]
vs    = 3000    # Wave velocity [m/s] 
ro0   = 2500    # Density [kg/m^3]
nt    = 2000    # Number of time steps
isx   = 500     # Source location [m] 
eps   = 0.5     # Stability limit
iplot = 20      # Snapshot frequency

dx = xmax/(nx-1)           # calculate space increment
x  = np.arange(0, nx)*dx   # initialize space coordinates
x  = np.transpose(x)

h = np.diff(x)  # Element sizes [m]

# parameters
ro = x*0 + ro0
mu = x*0 + ro*vs**2

# time step from stabiity criterion
dt = 0.5*eps*dx/np.max(np.sqrt(mu/ro))
# initialize time axis
t   = np.arange(1, nt+1)*dt  

# ---------------------------------------------------------------
# Initialize fields
# ---------------------------------------------------------------
u    = np.zeros(nx)
uold = np.zeros(nx)
unew = np.zeros(nx)

p    = np.zeros(nx)
pold = np.zeros(nx)
pnew = np.zeros(nx)

### 2. 源时间函数

在一维情况下，传播信号是源时间函数的积分。我们希望生成一个高斯波形，因此我们可以使用高斯函数的一阶导数来初始化源时间函数 $f(t)$。

\begin{equation}
f(t) = -\dfrac{2}{\sigma^2}(t - t_0)e^{-\dfrac{(t - t_0)^2}{\sigma^2}}
\end{equation}

初始化一个名为 'src' 的源时间函数。使用 $\sigma = 20 dt$ 作为高斯宽度，时间偏移 $t_0 = 3\sigma$。然后，将源在给定的图中可视化。"

In [4]:
# Initialization of the source time function
# ---------------------------------------------------------------
pt  = 20*dt     # Gaussian width
t0  = 3*pt      # Time shift
src = -2/pt**2 * (t-t0) * np.exp(-1/pt**2 * (t-t0)**2)

# Source vector
# ---------------------------------------------------------------
f = np.zeros(nx); f[isx:isx+1] = f[isx:isx+1] + 1.

# ---------------------------------------------------------------
# Plot source time fuction
# ---------------------------------------------------------------
plt.plot(t, src, color='b', lw=2, label='Source time function')
plt.ylabel('Amplitude', size=16)
plt.xlabel('time', size=16)
plt.legend()
plt.grid(True)
plt.show()

<IPython.core.display.Javascript object>

### 3. 质量矩阵

在实现所需的源函数后，现在我们初始化质量矩阵和刚度矩阵。通常，质量矩阵由以下公式给出：

\begin{equation}
M_{ij} = \int_{D} \rho \varphi_i \varphi_j dx = \int_{D_{\xi}} \rho \varphi_i \varphi_j d\xi
\end{equation}

接下来，我们引入定义的基函数，并进行一些代数处理，以得到质量矩阵的显式形式：

\begin{equation}
M_{ij} = \frac{\rho h}{6}
 \begin{pmatrix}
   \ddots  &    &    &    & 0\\
   1 & 4 &  1 &    &  \\
     & 1 &  4 &  1 &  \\
     &   &  1 &  4 & 1\\
   0 &   &    &    &  \ddots
 \end{pmatrix} 
\end{equation}"

In [5]:
# Mass matrix M_ij
# ---------------------------------------------------------------
M = np.zeros((nx,nx), dtype=float)
for i in range(1, nx-1):
    for j in range (1, nx-1):
        if j==i:
            M[i,j] = (ro[i-1]*h[i-1] + ro[i]*h[i])/3
        elif j==i+1:
            M[i,j] = ro[i]*h[i]/6
        elif j==i-1:
            M[i,j] = ro[i-1]*h[i-1]/6
        else:
            M[i,j] = 0
            
# Corner elements
M[0,0] = ro[0]*h[0]/3
M[nx-1,nx-1] = ro[nx-1]*h[nx-2]/3
# Invert M
Minv = np.linalg.inv(M)

### 4. 刚度矩阵

刚度矩阵的一般形式为

\begin{equation}
K_{ij} = \int_{D} \mu \partial_x\varphi_i \partial_x\varphi_j dx = \int_{D_{\xi}} \mu \partial_\xi\varphi_i \partial_\xi\varphi_j d\xi
\end{equation} 

经过一些代数处理后，我们得到刚度矩阵的显式形式为

\begin{equation}
K_{ij} = \frac{\mu}{h}
 \begin{pmatrix}
   \ddots  &    &    &    & 0\\
  -1 & 2 & -1 &    &  \\
     &-1 &  2 & -1 &  \\
     &   & -1 &  2 & -1\\
   0 &   &    &    &  \ddots
 \end{pmatrix} 
\end{equation}

In [6]:
# Stiffness matrix Kij
# ---------------------------------------------------------------
K = np.zeros((nx,nx), dtype=float)
for i in range(1, nx-1):
    for j in range(1, nx-1):
        if i==j:
            K[i,j] = mu[i-1]/h[i-1] + mu[i]/h[i]
        elif i==j+1:
            K[i,j] = -mu[i-1]/h[i-1]
        elif i+1==j:
            K[i,j] = -mu[i]/h[i]
        else:
            K[i,j] = 0

K[0,0] = mu[0]/h[0]
K[nx-1,nx-1] = mu[nx-1]/h[nx-2]

In [7]:
# Display finite element matrices
# ---------------------------------------------------------------
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(K[1:10,1:10])
ax1.set_title('Finite-Element Stiffness Matrix $\mathbf{K}$')
ax1.axis("off")

ax2.imshow(Minv[1:10,1:10])
ax2.set_title('Finite-Element Mass Matrix $\mathbf{M^{-1}}$')
ax2.axis("off")

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

### 5. 有限差分矩阵

我们通过矩阵-向量操作实现有限差分方案，以便与有限元解进行比较。

经典的二阶导数矩阵（对应于有限元方案中的刚度矩阵）可以初始化为

\begin{equation}
D_{ij} = \frac{\mu}{dx^2}
 \begin{pmatrix}
   -2 & 1  &    &    &  \\
    1 & -2 & 1  &    &  \\
      &  & \ddots  &  & \\
      &   & 1 &  -2 & 1\\
      &   &    &  1 & -2
 \end{pmatrix} 
\end{equation}

为了将有限差分方案形式上与有限元方案相一致，我们还需要对应的质量矩阵，简单使用倒数密度即可，我们初始化为

\begin{equation}
M^{fd}_{ij} = 
 \begin{pmatrix}
   1/{\rho_1} &   &    &    &  \\
     &  1/{\rho_2} &  &    &  \\
      &  & \ddots  &  & \\
      &   &   &  1/{\rho_{N-1}}  & \\
      &   &    &   & 1/{\rho_{N}}
 \end{pmatrix} 
\end{equation}

In [8]:
# Initialize finite differences matrices 
# ---------------------------------------------------------------
Mfd = np.zeros((nx,nx), dtype=float)
D  = np.zeros((nx,nx), dtype=float)
dx = h[1]

for i in range(nx):
    Mfd[i,i] = 1./ro[i]
    if i>0:
        if i<nx-1:
            D[i+1,i] =1
            D[i-1,i] =1
            D[i,i] = -2
            
D = ro0 * vs**2 * D/dx**2

In [9]:
# Display differences matrices
# ---------------------------------------------------------------
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(-D[1:10,1:10])
ax1.set_title('Finite-Difference Derivative Matrix $\mathbf{D}$')
ax1.axis("off")

ax2.imshow(Mfd[1:10,1:10])
ax2.set_title('Finite-Difference Mass Matrix $\mathbf{M_{fd}}$')
ax2.axis("off")

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

### 6. 有限元解

最后，我们使用计算得到的质量矩阵 $M$ 和刚度矩阵 $K$，结合有限差分外推方案实现有限元解

\begin{equation}
\mathbf{u}(t + dt) = dt^2 (\mathbf{M}^T)^{-1}[\mathbf{f} - \mathbf{K}^T\mathbf{u}] + 2\mathbf{u} - \mathbf{u}(t-dt).
\end{equation}

In [11]:
# Initialize animated plot
# ---------------------------------------------------------------
plt.figure(figsize=(12,4))

line1 = plt.plot(x, u, 'k', lw=1.5, label='FEM')
line2 = plt.plot(x, p, 'r', lw=1.5, label='FDM')
plt.title('Finite elements 1D Animation', fontsize=16)
plt.ylabel('Amplitude', fontsize=12)
plt.xlabel('x (m)', fontsize=12)

plt.ion()   # set interective mode
plt.show()

<IPython.core.display.Javascript object>

In [12]:
# ---------------------------------------------------------------
# Time extrapolation
# ---------------------------------------------------------------
for it in range(nt):
    # --------------------------------------
    # Finite Element Method
    unew = (dt**2) * Minv @ (f*src[it]  -  K @ u) + 2*u - uold                         
    uold, u = u, unew
    
    # --------------------------------------
    # Finite Difference Method
    pnew = (dt**2) * Mfd @ ( f/dx*src[it]+ D @ p) + 2*p - pold
    pold, p = p, pnew
     
    # --------------------------------------   
    # Animation plot. Display both solutions
    if not it % iplot:
        for l in line1:
            l.remove()
            del l
        for l in line2:
            l.remove()
            del l
        line1 = plt.plot(x, u, 'k', lw=1.5, label='FEM')
        line2 = plt.plot(x, p, 'r', lw=1.5, label='FDM')
        plt.legend()
        plt.gcf().canvas.draw()

<IPython.core.display.Javascript object>